In [1]:
import pandas as pd
import numpy as np
import psycopg2
from tqdm import tqdm
import json

In [2]:
import os
from google.cloud import translate_v3beta1 as translate

In [3]:
!curl -O https://dl.google.com/dl/cloudsdk/channels/rapid/downloads/google-cloud-cli-407.0.0-linux-x86_64.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  155M  100  155M    0     0  55.6M      0  0:00:02  0:00:02 --:--:-- 55.5M


In [4]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/content/drive/MyDrive/yeoreodigm/data_files/google_api_key.json"

In [5]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS']

'/content/drive/MyDrive/yeoreodigm/data_files/google_api_key.json'

In [8]:
!pip install google-cloud-translate==2.0.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached google_cloud_translate-2.0.1-py2.py3-none-any.whl (90 kB)
  Using cached google_cloud_core-1.7.3-py2.py3-none-any.whl (28 kB)
  Attempting uninstall: google-cloud-core
    Found existing installation: google-cloud-core 1.0.3
    Uninstalling google-cloud-core-1.0.3:
      Successfully uninstalled google-cloud-core-1.0.3
  Attempting uninstall: google-cloud-translate
    Found existing installation: google-cloud-translate 1.5.0
    Uninstalling google-cloud-translate-1.5.0:
      Successfully uninstalled google-cloud-translate-1.5.0


In [6]:
!gcloud auth application-default print-access-token

ya29.c.b0Aa9VdykSyba4Erzg4gXH7SQg81s6mJ60kNE6dclgwMGkUMMplXmMJY9ojYx5pE1jMheZzIy2zptqBooBUvpuYR1RAFwXRBUXKirNcDMRW7LvqtDyF0g6EgfOCwIl8xn8FZ5rfa4XtNRen2iq2sFQaBN_2oeUVU80pSmI4z4lBmcymSRCGVbbkKScpg-gPcqYCV3dQsNYhs_5nnx7bnf-4wrbeN0jM8o................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [7]:
def load_db():
  with open("/content/drive/MyDrive/yeoreodigm/data_files/db_info.json") as json_file:
    DB_INFO = json.load(json_file)
  
  endpoint = DB_INFO["ENDPOINT"]
  dbname = DB_INFO["DB_NAME"]
  user = DB_INFO["USER_ID"]
  password = DB_INFO["PASSWORD"]
  db = psycopg2.connect(host=endpoint,dbname=dbname,user=user,password=password)
  return db

In [61]:
db=load_db()
cursor =db.cursor()
cursor.execute("select * from places_extra_info")
result = cursor.fetchall()
extra_info = pd.DataFrame(result)

In [62]:
extra_info.head(2)

,0,1,2,3,4,5,6,7,8,9,10
0,23,,,,,,,,,,23
1,51,"하절기(4~10월) 08:00~16:00, 동절기(11~3월) 09:00~16:00",,"평일 : 08:00 ~ 16:00 , 주말 : 08:00 ~ 16:00","유료,어른 1,000원, 청소년 600원, 유아.장애인, 무료",실외,"기타, 숲길 , 산책로","공용주차장,화장실",상,,51


In [63]:
extra_info.columns = ['id','detail_info','extra_info','hours','fee','indoor/outdoor','purpose','amenity','difficulty','estimated_time','id2']
extra_info.head(2)

,id,detail_info,extra_info,hours,fee,indoor/outdoor,purpose,amenity,difficulty,estimated_time,id2
0,23,,,,,,,,,,23
1,51,"하절기(4~10월) 08:00~16:00, 동절기(11~3월) 09:00~16:00",,"평일 : 08:00 ~ 16:00 , 주말 : 08:00 ~ 16:00","유료,어른 1,000원, 청소년 600원, 유아.장애인, 무료",실외,"기타, 숲길 , 산책로","공용주차장,화장실",상,,51


In [64]:
extra_info.isnull().sum()

id                0
detail_info       0
extra_info        0
hours             0
fee               0
indoor/outdoor    0
purpose           0
amenity           0
difficulty        0
estimated_time    0
id2               0
dtype: int64

In [65]:
extra_info = extra_info[['id','detail_info','extra_info','hours','fee','indoor/outdoor','purpose','amenity','difficulty','estimated_time']]

###영어 : "en-US" / 중국어(간체) : "zh-CN"

In [57]:
from google.cloud import translate
def translate_text(text,target_lang):
  
  if text == '':
    return ''

  project_id="proven-env-367507"
  client = translate.TranslationServiceClient()
  location = "global"
  parent = f"projects/{project_id}/locations/{location}"


  response = client.translate_text(parent= parent,contents= [text],mime_type= "text/plain", source_language_code="ko",
                                    target_language_code = target_lang
  )
  return response.translations[0].translated_text
  

In [29]:
translate_text('운영시간 : 10시~6시','en-US')

'Operating hours: 10 am to 6 pm'

In [19]:
translate_text('운영시간 : 10시~6시','zh-CN')

'营业时间：上午 10 点至下午 6 点'

In [32]:
extra_info.head(2)

,id,detail_info,extra_info,hours,fee,indoor/outdoor,purpose,amenity,difficulty,estimated_time
0,23,,,,,,,,,
1,51,"하절기(4~10월) 08:00~16:00, 동절기(11~3월) 09:00~16:00",,"평일 : 08:00 ~ 16:00 , 주말 : 08:00 ~ 16:00","유료,어른 1,000원, 청소년 600원, 유아.장애인, 무료",실외,"기타, 숲길 , 산책로","공용주차장,화장실",상,


In [42]:
extra_info = extra_info.sort_values(by = 'id',ascending=True)
extra_info_eng = extra_info.copy()
extra_info_cn = extra_info.copy()

##영문으로 번역

In [67]:
extra_info_eng['detail_info'] = extra_info_eng['detail_info'].apply(lambda x: translate_text(x,'en-US'))

In [68]:
extra_info_eng['extra_info'] = extra_info_eng['extra_info'].apply(lambda x: translate_text(x,'en-US'))
extra_info_eng['hours'] = extra_info_eng['hours'].apply(lambda x: translate_text(x,'en-US'))
extra_info_eng['fee'] = extra_info_eng['fee'].apply(lambda x: translate_text(x,'en-US'))
extra_info_eng['indoor/outdoor'] = extra_info_eng['indoor/outdoor'].apply(lambda x: translate_text(x,'en-US'))

In [69]:
extra_info_eng['purpose'] = extra_info_eng['purpose'].apply(lambda x: translate_text(x,'en-US'))
extra_info_eng['amenity'] = extra_info_eng['amenity'].apply(lambda x: translate_text(x,'en-US'))
extra_info_eng['difficulty'] = extra_info_eng['difficulty'].apply(lambda x: translate_text(x,'en-US'))
extra_info_eng['estimated_time'] = extra_info_eng['estimated_time'].apply(lambda x: translate_text(x,'en-US'))

###번역 확인 및 max length 확인

In [70]:
extra_info_eng.sample(5)

,id,detail_info,extra_info,hours,fee,indoor/outdoor,purpose,amenity,difficulty,estimated_time
828,818,,,,,,,public parking lot,,
1070,1071,It is open daily from 08:00 to 19:00 and is op...,,"Weekdays : 08:00 ~ 19:00 , Weekends : 08:00 ~ ...","Paid, experience diving: 100,000 won",indoor + outdoor,,"Cash payment, credit card payment, restroom, f...",under,2-3 hours
324,295,Reservations are required for all experience p...,,"Weekdays : 09:00 ~ 17:00 , Weekends : 09:00 ~ ...","N/A, barley pizza experience 7,000 won per per...",,,,,
788,769,"Subject to change according to the season, roo...",,"Weekdays : 06:00 ~ 20:30 , Weekends : 06:00 ~ ...",,,,"Public parking lot, cash payment, card payment...",,
1084,1085,* Monday regular holiday,,"Weekdays : 09:00 ~ 18:00 , Weekends : 09:00 ~ ...",,,,,,


In [82]:
extra_info_eng['len'] = extra_info_eng['estimated_time'].apply(lambda x: len(x))
extra_info_eng.sort_values(by='len',ascending=False).head(2)

,id,detail_info,extra_info,hours,fee,indoor/outdoor,purpose,amenity,difficulty,estimated_time,len
58,27,Entry and exit may be controlled depending on ...,,,"Free, paid parking.",outdoor,,,,more than 3 hours,17
1048,1049,Regular holidays: 2nd and 4th Sundays Irregula...,,"Weekdays : 10:00 ~ 19:00 , Weekends : 10:00 ~ ...","Paid, different by product",indoor + outdoor,"Climbing, camping, experience, water play, pho...","Cash payment, credit card payment, toilet, fre...",under,more than 3 hours,17


##중문으로 번역

In [71]:
extra_info_cn['detail_info'] = extra_info_cn['detail_info'].apply(lambda x: translate_text(x,'zh-CN'))

In [72]:
extra_info_cn['extra_info'] = extra_info_cn['extra_info'].apply(lambda x: translate_text(x,'zh-CN'))
extra_info_cn['hours'] = extra_info_cn['hours'].apply(lambda x: translate_text(x,'zh-CN'))
extra_info_cn['fee'] = extra_info_cn['fee'].apply(lambda x: translate_text(x,'zh-CN'))
extra_info_cn['indoor/outdoor'] = extra_info_cn['indoor/outdoor'].apply(lambda x: translate_text(x,'zh-CN'))

In [73]:
extra_info_cn['purpose'] = extra_info_cn['purpose'].apply(lambda x: translate_text(x,'zh-CN'))
extra_info_cn['amenity'] = extra_info_cn['amenity'].apply(lambda x: translate_text(x,'zh-CN'))
extra_info_cn['difficulty'] = extra_info_cn['difficulty'].apply(lambda x: translate_text(x,'zh-CN'))
extra_info_cn['estimated_time'] = extra_info_cn['estimated_time'].apply(lambda x: translate_text(x,'zh-CN'))

###번역 확인 및 max length 확인

In [83]:
extra_info_cn.sample(5)

,id,detail_info,extra_info,hours,fee,indoor/outdoor,purpose,amenity,difficulty,estimated_time
111,78,全年无休 / 18:00 售票截止 / 体验时间 09:00-17:30,优秀的旅游业务,"平日 : 09:00 ~ 19:00 , 周末 : 09:00 ~ 19:00","有偿，成人 11,000 韩元，残疾人，老年人，青少年 9,000 韩元",室内+室外,拍照点、吉他、表演/展览,公共停车场、现金支付、刷卡支付、卫生间、免费WIFI、吸烟区、便利店、饮料站、指导和信息设施...,在下面,1-2小时
559,531,,,,,,,,,
228,196,全年无休 / 入场时间：至 17:00 / 允许露营（呼气预约 https://www.fo...,,"平日 : 09:00 ~ 18:00 , 周末 : 09:00 ~ 18:00",付费，成人1.000韩元，青少年600韩元，儿童300韩元，老年人和残疾人免费（1-3年级最...,户外的,远足，体验,公共停车场、现金支付、刷卡支付、洗手间、免费WIFI、便利店、饮料站、指导和信息设施、警报和...,中间,1-2小时
327,298,,,,,,,公共停车场,,
186,150,全年无休/生日当天免费入场/每月最后一个周一闭馆/两种礼堂套餐价格有效/最后入场时间17:00,,"平日 : 09:00 ~ 18:00 , 周末 : 09:00 ~ 18:00","付费，成人9,000韩元，青少年和儿童8,000韩元，老年人和残疾人7,000韩元（特丽爱博...",室内+室外,表演/展览、拍照点,公共停车场、现金支付、刷卡支付、厕所、便利店、指导和信息设施、警报和疏散设施,在下面,1-2小时


In [95]:
extra_info_cn['len'] = extra_info_cn['estimated_time'].apply(lambda x: len(x))
extra_info_cn.sort_values(by='len',ascending=False).head(2)

,id,detail_info,extra_info,hours,fee,indoor/outdoor,purpose,amenity,difficulty,estimated_time,len
398,370,全年无休 / 09:00 ~ 22:00（最后售票时间 21:00） / 可携带宠物进出,优秀的旅游业务,"平日 : 09:00 ~ 22:00 , 周末 : 09:00 ~ 22:00","有偿，成人13,000韩元，青少年11,000韩元，儿童10,000韩元，老年人和残疾儿童同价。",室内+室外,,公共停车场、现金支付、刷卡支付、卫生间、免费WIFI、吸烟区、指导和信息设施、警报和疏散设施,在下面,1-2小时,5
204,172,休馆日：每周一（如遇节假日则次日休息）、每年1月1日、元旦、中秋,,"平日 : 09:00 ~ 18:00 , 周末 : 09:00 ~ 18:00",自由的,里面,表演/展览,公共停车场、卫生间、免费WIFI、吸烟区、饮料站、指导和信息设施、警报和疏散设施、电梯,在下面,1-2小时,5


##DB 삽입

###영문 데이터

In [113]:
db.rollback()

In [111]:
extra_info_eng['id'] = extra_info_eng['id'].apply(lambda x: int(x))
extra_info_cn['id'] = extra_info_cn['id'].apply(lambda x: int(x))

In [114]:
for i in tqdm(range(1089)):
  now = extra_info_eng.iloc[i]
  val = now.values
  # print(val[0],val[1],val[2],val[3],val[4],val[5],val[6],val[7],val[8],val[9])
  # break
  cursor.execute("INSERT INTO places_extra_info_en(place_id, detail_info, extra_info, operating_hours, fee, indoor_outdoor, purpose, amenities, difficulty, estimated_time)\
   VALUES(%s, %s, %s, %s, %s,%s, %s, %s, %s, %s)",(int(val[0]),val[1],val[2],val[3],val[4],val[5],val[6],val[7],val[8],val[9]))

100%|██████████| 1089/1089 [03:31<00:00,  5.16it/s]


In [115]:
db.commit()

###중국어 데이터

In [117]:
for i in tqdm(range(1089)):
  now = extra_info_cn.iloc[i]
  val = now.values
  cursor.execute("INSERT INTO places_extra_info_zh(place_id, detail_info, extra_info, operating_hours, fee, indoor_outdoor, purpose, amenities, difficulty, estimated_time)\
   VALUES(%s, %s, %s, %s, %s,%s, %s, %s, %s, %s)",(int(val[0]),val[1],val[2],val[3],val[4],val[5],val[6],val[7],val[8],val[9]))

100%|██████████| 1089/1089 [03:31<00:00,  5.16it/s]


In [118]:
db.commit()